In [1]:
# make a predict based on EGF only

In [2]:
import pandas as pd
import glob
import numpy as np
from scipy import stats
from sklearn import cluster
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score



In [3]:
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [688]:
###### template to fill
filename = './median_phospho_data.csv' # use your path
df1 = pd.read_csv(filename, index_col=None, header=0)

In [689]:
df1.drop(['p.HER2','p.PLCg2'], axis=1,inplace=True)

In [691]:
df=df1[df1['treatment']=="full"]

In [692]:
df

,cell_line,treatment,time,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,p.4EBP1,...,p.p90RSK,p.PDPK1,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
10,184A1,full,0.0,1.268450,3.011949,3.269844,2.971663,6.726300,3.876879,2.905013,...,3.156479,2.841768,6.638407,6.080130,2.677548,2.477925,2.320824,2.887623,1.565863,2.264294
49,184B5,full,0.0,0.644142,2.724148,1.992426,2.214774,6.145829,2.379644,2.210643,...,2.703306,2.187006,5.049717,4.743696,2.058578,2.113998,2.192451,2.412290,1.562300,2.227544
81,AU565,full,0.0,0.677910,1.779176,1.673790,1.759997,6.159892,3.368162,2.533022,...,4.152506,1.040180,5.587673,NaN,2.531707,1.647424,1.839102,2.551210,1.107368,2.433133
120,BT20,full,0.0,0.981176,2.298729,2.604696,2.357667,6.357916,3.449260,2.922381,...,3.965937,1.474105,6.637675,6.917825,2.257894,2.179263,2.762720,3.184696,2.611055,3.028543
159,BT474,full,0.0,1.063666,2.434756,1.759334,3.102078,6.121343,2.168054,2.905236,...,2.842842,2.185061,4.202805,6.846955,2.035517,1.780411,2.359247,3.278507,1.167880,2.727995
198,BT483,full,0.0,0.796624,1.588045,1.968335,2.244023,5.968479,2.542089,2.245383,...,2.521351,1.548628,3.044003,5.756088,2.066058,1.447193,2.560634,3.132175,3.348869,2.698918
230,BT549,full,0.0,0.596065,2.180474,3.582375,1.987784,7.607806,3.513793,2.576371,...,2.110591,0.678260,2.820260,6.863532,1.375215,1.977462,2.114881,2.977174,2.351837,2.616188
259,CAL120,full,0.0,0.932206,2.579209,3.434786,3.019115,6.666223,4.228441,3.644402,...,2.801621,1.860126,5.718332,5.378289,1.961118,2.054174,2.538838,3.059213,3.460249,2.758271
270,CAL148,full,0.0,1.245810,1.805119,2.649700,3.482292,5.679523,3.643943,4.162734,...,3.243798,1.967921,2.975085,4.516129,2.200839,1.562807,2.732056,2.928218,1.387870,1.894472
309,CAL51,full,0.0,1.118627,2.552589,4.744935,3.550320,10.515575,5.618507,3.328763,...,4.195468,2.098155,7.833360,7.639304,1.787332,1.891504,2.248662,2.635630,2.119403,1.888591


In [693]:
c1=df.loc[df.cell_line == 'CAL120']
c1=c1.append(df.loc[df.cell_line == 'CAMA1' ])
c1=c1.append(df.loc[df.cell_line == 'HCC1143' ])
c1=c1.append(df.loc[df.cell_line == 'KPL1' ])
c1=c1.append(df.loc[df.cell_line == 'ZR75B' ])

In [694]:
c1

,cell_line,treatment,time,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,p.4EBP1,...,p.p90RSK,p.PDPK1,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
259,CAL120,full,0.0,0.932206,2.579209,3.434786,3.019115,6.666223,4.228441,3.644402,...,2.801621,1.860126,5.718332,5.378289,1.961118,2.054174,2.538838,3.059213,3.460249,2.758271
377,CAMA1,full,0.0,0.680420,1.919718,1.937110,2.709673,5.923626,3.306433,3.076086,...,1.758234,1.675660,5.920553,6.877702,1.580071,1.539478,1.757436,2.797769,1.217047,2.439276
573,HCC1143,full,0.0,1.068090,2.797386,2.656847,3.519858,6.533675,4.912958,3.994795,...,2.727622,2.134740,6.974143,6.759208,2.256158,1.885630,3.631515,3.456057,2.901550,3.161714
1326,KPL1,full,0.0,1.240168,2.965467,3.654952,3.280202,6.980764,4.710842,3.448468,...,3.805560,2.901748,7.093953,5.718664,2.150253,2.363313,1.606715,3.028408,1.303958,1.307696
2338,ZR75B,full,0.0,0.793309,2.136842,2.531233,3.009793,6.140302,3.662012,3.257238,...,2.951726,1.733651,5.768492,6.658976,1.935627,1.793562,2.861304,2.947758,2.593895,3.056968


In [695]:
# drop some times 'CAL120', 'CAMA1', 'HCC1143', 'KPL1', 'ZR75B'
indexNames =df[ df['cell_line'] == 'CAL120' ].index
indexNames2=df[ df['cell_line'] == 'CAMA1' ].index
indexNames3=df[ df['cell_line'] == 'HCC1143' ].index
indexNames4=df[ df['cell_line'] == 'KPL1' ].index
indexNames5=df[ df['cell_line'] == 'ZR75B' ].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.drop(indexNames2 , inplace=True)
df.drop(indexNames3 , inplace=True)
df.drop(indexNames4 , inplace=True)
df.drop(indexNames5 , inplace=True)


In [696]:
# fill na
df.fillna(method='ffill', inplace=True)
df.drop(['treatment','time'], axis=1,inplace=True)
df.set_index('cell_line',inplace=True)

In [697]:
# fill na
c1.fillna(method='ffill', inplace=True)
c1.drop(['treatment','time'], axis=1,inplace=True)
c1.set_index('cell_line',inplace=True)

In [708]:
diff_df = df - c1.iloc[0, :]
norm_df = diff_df.apply(np.linalg.norm, axis=1)
df.loc[norm_df.idxmin()]

b.CATENIN        0.926847
cleavedCas       3.086199
CyclinB          3.397483
GAPDH            3.183370
IdU              6.264556
Ki.67            4.405243
p.4EBP1          3.053248
p.Akt.Ser473.    3.197812
p.AKT.Thr308.    2.679626
p.AMPK           2.931494
p.BTK            2.769968
p.CREB           0.471780
p.ERK            2.578254
p.FAK            2.110050
p.GSK3b          2.507239
p.H3             2.938720
p.JNK            1.995832
p.MAP2K3         1.787723
p.MAPKAPK2       1.987924
p.MEK            2.977682
p.MKK3.MKK6      2.292747
p.MKK4           3.815610
p.NFkB           3.145801
p.p38            2.073899
p.p53            1.613947
p.p90RSK         3.115411
p.PDPK1          2.357047
p.RB             6.441925
p.S6             6.291298
p.S6K            2.446044
p.SMAD23         2.274946
p.SRC            2.148126
p.STAT1          2.980237
p.STAT3          2.261184
p.STAT5          2.506966
Name: MCF10F, dtype: float64

In [709]:
norm_df.sort_values()

cell_line
MCF10F         3.107827
MDAMB231       3.267748
EFM19          3.376684
HCC1187        3.406429
184A1          3.425499
HCC2185        3.689751
HDQP1          3.778698
MACLS2         3.780536
HBL100         3.805440
EVSAT          3.870308
HCC1428        4.045111
MFM223         4.050887
ZR7530         4.054030
BT20           4.060089
MPE600         4.087968
HCC2157        4.124287
MCF12A         4.163062
MCF10A         4.235542
HCC38          4.239381
EFM192A        4.247901
T47D           4.315467
HCC1395        4.320621
HCC1599        4.338613
HCC1937        4.345652
CAL851         4.357174
HCC70          4.388081
MDAMB134VI     4.399281
UACC3199       4.444375
ZR751          4.486204
184B5          4.554240
                 ...   
MDAMB415       4.625130
Hs578T         4.660216
JIMT1          4.708480
MDAMB157       4.728060
HCC1500        4.736486
MDAMB361       4.827330
AU565          4.974569
HCC1806        5.028115
BT474          5.054621
BT483          5.063761
BT549 

In [710]:
t1=df1.loc[df1.cell_line == 'MCF10F']
indexNames=t1[ t1['treatment'] == 'full' ].index
# Delete these row indexes from dataFrame
t1.drop(indexNames , inplace=True)

In [711]:
t1.drop(['treatment','time', 'cell_line'], axis=1,inplace=True)


In [712]:
result=t1

In [713]:
t2=df1.loc[df1.cell_line == 'UACC893']
indexNames=t2[ t2['treatment'] == 'full' ].index
# Delete these row indexes from dataFrame
t2.drop(indexNames , inplace=True)
t2.drop(['treatment','time', 'cell_line'], axis=1,inplace=True)

In [714]:
result=result.append(t2)

In [715]:
t3=df1.loc[df1.cell_line == 'BT20']
indexNames=t3[ t3['treatment'] == 'full' ].index
# Delete these row indexes from dataFrame
t3.drop(indexNames , inplace=True)
t3.drop(['treatment','time', 'cell_line'], axis=1,inplace=True)

In [716]:
result=result.append(t3)

In [717]:
t4=df1.loc[df1.cell_line == '184A1']
indexNames=t4[ t4['treatment'] == 'full' ].index
# Delete these row indexes from dataFrame
t4.drop(indexNames , inplace=True)
t4.drop(['treatment','time', 'cell_line'], axis=1,inplace=True)

In [718]:
result=result.append(t4)

In [719]:
t5=df1.loc[df1.cell_line == 'HCC38']
indexNames=t5[ t5['treatment'] == 'full' ].index
# Delete these row indexes from dataFrame
t5.drop(indexNames , inplace=True)
t5.drop(['treatment','time', 'cell_line'], axis=1,inplace=True)

In [720]:
result=result.append(t5)

In [586]:
###### template to fill  
#'184A1', 'HCC2157', 'MCF10A', 'MCF10F', 'MDAkb2' those cell are not to predict
filename = './subchallenge_4_template_data.csv' # use your path
df_temp4 = pd.read_csv(filename, index_col=None, header=0)

In [587]:
df_temp4

,cell_line,treatment,time,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,p.4EBP1,...,p.p90RSK,p.PDPK1,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
0,CAL120,EGF,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CAL120,EGF,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CAL120,EGF,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CAL120,EGF,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CAL120,EGF,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CAL120,EGF,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CAL120,EGF,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CAL120,EGF,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CAL120,EGF,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,CAL120,EGF,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [640]:
df_temp4[df_temp4['cell_line']== 'CAL120']

,cell_line,treatment,time,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,p.4EBP1,...,p.p90RSK,p.PDPK1,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
0,CAL120,EGF,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CAL120,EGF,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CAL120,EGF,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CAL120,EGF,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CAL120,EGF,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CAL120,EGF,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CAL120,EGF,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CAL120,EGF,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CAL120,EGF,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,CAL120,EGF,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [684]:
df_temp4.columns

Index(['cell_line', 'treatment', 'time', 'b.CATENIN', 'cleavedCas', 'CyclinB',
       'GAPDH', 'IdU', 'Ki.67', 'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.',
       'p.AMPK', 'p.BTK', 'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3',
       'p.JNK', 'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4',
       'p.NFkB', 'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.RB', 'p.S6',
       'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5'],
      dtype='object')

In [685]:
df1.columns

Index(['cell_line', 'treatment', 'time', 'b.CATENIN', 'cleavedCas', 'CyclinB',
       'GAPDH', 'IdU', 'Ki.67', 'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.',
       'p.AMPK', 'p.BTK', 'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3',
       'p.HER2', 'p.JNK', 'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6',
       'p.MKK4', 'p.NFkB', 'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.PLCg2',
       'p.RB', 'p.S6', 'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3',
       'p.STAT5'],
      dtype='object')

In [721]:
result

,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,p.4EBP1,p.Akt.Ser473.,p.AKT.Thr308.,p.AMPK,...,p.p90RSK,p.PDPK1,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
1444,0.855540,2.499864,1.399830,2.493123,4.660937,1.948277,1.586645,1.983654,2.093340,3.032344,...,2.637643,1.690649,3.314925,4.632658,1.679686,1.725113,1.542561,2.017830,1.850496,1.898787
1445,0.950447,2.521193,1.515691,2.715733,4.789183,1.913723,2.208299,2.934302,2.607699,3.082060,...,3.307665,1.966866,3.533035,4.420281,2.040544,1.852232,1.741494,2.184899,2.057786,2.176713
1446,0.861342,2.430443,1.442294,2.736052,4.802800,1.892977,2.207018,3.015345,2.455299,3.051869,...,3.342001,1.972753,3.478702,4.859043,2.061273,1.730003,1.693360,2.180289,1.987825,2.233488
1447,0.647582,2.176822,1.284884,2.315288,4.756889,1.757489,1.969192,2.754697,2.144004,2.914260,...,3.067155,1.623610,3.265652,5.287989,1.920690,1.494569,1.561315,2.088229,1.675875,2.098096
1448,0.870495,2.419352,1.465950,2.566252,4.733941,1.918355,2.250648,2.729778,2.396156,2.886512,...,3.317780,1.735350,3.344337,5.645933,2.021957,1.751453,1.658747,2.287379,1.831223,2.087075
1449,0.985165,2.564084,1.580554,2.804204,4.797698,1.993500,2.369868,2.748138,2.505193,2.736549,...,3.390166,1.906487,3.546853,5.745845,2.164402,1.914123,1.813935,2.446129,1.900999,2.207512
1450,0.935766,2.498757,1.567218,2.668521,4.810053,1.897568,2.275418,2.719143,2.470301,2.460884,...,3.277571,1.845911,3.597943,5.548917,2.208931,1.902340,1.878566,2.526993,1.715969,2.196945
1451,0.894831,2.431745,1.532946,2.599234,4.776610,1.814994,2.102516,2.657104,2.413779,2.406427,...,3.207218,1.787773,3.655597,5.441046,2.175834,1.864358,1.848767,2.553951,1.571765,2.139599
1452,1.007206,2.493612,1.600453,2.782575,4.758566,1.878306,2.155713,2.694587,2.505768,2.516863,...,3.270909,1.842478,3.757710,5.595831,2.181947,1.957455,1.860764,2.657198,1.607279,2.137212
1453,1.095788,2.543251,1.645183,2.917793,4.735015,1.967822,2.266639,2.759606,2.571603,2.612392,...,3.276263,1.874863,3.771149,5.790127,2.184306,2.016146,1.875931,2.744061,1.634118,2.155515


In [722]:
result.insert (0, "cell_line", df_temp4.cell_line.values)

In [723]:
result.insert (1, "treatment", df_temp4.treatment.values)

In [724]:
result.insert (2, "time", df_temp4.time.values)

In [727]:
result.reset_index(inplace=True)

In [728]:
result

,index,cell_line,treatment,time,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,...,p.p90RSK,p.PDPK1,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
0,1444,CAL120,EGF,0.0,0.855540,2.499864,1.399830,2.493123,4.660937,1.948277,...,2.637643,1.690649,3.314925,4.632658,1.679686,1.725113,1.542561,2.017830,1.850496,1.898787
1,1445,CAL120,EGF,5.5,0.950447,2.521193,1.515691,2.715733,4.789183,1.913723,...,3.307665,1.966866,3.533035,4.420281,2.040544,1.852232,1.741494,2.184899,2.057786,2.176713
2,1446,CAL120,EGF,7.0,0.861342,2.430443,1.442294,2.736052,4.802800,1.892977,...,3.342001,1.972753,3.478702,4.859043,2.061273,1.730003,1.693360,2.180289,1.987825,2.233488
3,1447,CAL120,EGF,9.0,0.647582,2.176822,1.284884,2.315288,4.756889,1.757489,...,3.067155,1.623610,3.265652,5.287989,1.920690,1.494569,1.561315,2.088229,1.675875,2.098096
4,1448,CAL120,EGF,13.0,0.870495,2.419352,1.465950,2.566252,4.733941,1.918355,...,3.317780,1.735350,3.344337,5.645933,2.021957,1.751453,1.658747,2.287379,1.831223,2.087075
5,1449,CAL120,EGF,17.0,0.985165,2.564084,1.580554,2.804204,4.797698,1.993500,...,3.390166,1.906487,3.546853,5.745845,2.164402,1.914123,1.813935,2.446129,1.900999,2.207512
6,1450,CAL120,EGF,23.0,0.935766,2.498757,1.567218,2.668521,4.810053,1.897568,...,3.277571,1.845911,3.597943,5.548917,2.208931,1.902340,1.878566,2.526993,1.715969,2.196945
7,1451,CAL120,EGF,30.0,0.894831,2.431745,1.532946,2.599234,4.776610,1.814994,...,3.207218,1.787773,3.655597,5.441046,2.175834,1.864358,1.848767,2.553951,1.571765,2.139599
8,1452,CAL120,EGF,40.0,1.007206,2.493612,1.600453,2.782575,4.758566,1.878306,...,3.270909,1.842478,3.757710,5.595831,2.181947,1.957455,1.860764,2.657198,1.607279,2.137212
9,1453,CAL120,EGF,60.0,1.095788,2.543251,1.645183,2.917793,4.735015,1.967822,...,3.276263,1.874863,3.771149,5.790127,2.184306,2.016146,1.875931,2.744061,1.634118,2.155515


In [729]:
result.to_csv(r'subchallenge_4_v.csv')